la cosa ora "funziona" ma non riesco a far ritornare la pagina principale dei risultati, ritorna solo html nudo e crudo

In [ ]:
#con yield che ritorna parte della funzione (versione primitiva)

from flask import Flask, request, render_template, Response
from processing import kw_ranking, crawler, cleaner

app = Flask(__name__)
app.config["DEBUG"] = True

#render about page 
@app.route('/about', methods = ["GET"])
def about():
    return render_template("about.html")

#render main page
@app.route('/', methods = ["GET", "POST"])
def input_page():
    errors = ""
    output = ""
    if request.method == "POST":
        url = None
        url = f"https://www.{request.form['url']}"
      
        if url is not None:
            try:
                
                def generate():
                    yield "<br/>"
                    yield str(crawler(url))
                    links = crawler(url)
                    yield str(kw_ranking(links))
                    output = kw_ranking(links)  
                return Response(generate(), mimetype='text\html')
                                           
                #render_template("results.html", result=output)
            
            except Exception as e:
                #errors = "🐸 The anti-bug frog has caught something! Can you try again with another site? Sorry, it's still a beta version!"
                errors = e
    return render_template("main.html", errors=errors)

if __name__ == '__main__':
    app.run(use_reloader=False)


Da qui in avanti vari e diversi esperimenti apparentemente non fruttuosi

In [ ]:
def generator():
    yield "process started"
    time.sleep(5)
    yield "process going"
    time.sleep(5)
    yield "process finished"

for i in generator ():
    print (i)

In [ ]:
#con yield che ritorna parte della funzione (versione primitiva)

from flask import Flask, request, render_template, Response
from processing import kw_ranking, crawler, cleaner

app = Flask(__name__)
app.config["DEBUG"] = True

#render about page 
@app.route('/about', methods = ["GET"])
def about():
    return render_template("about.html")

#render main page
@app.route('/', methods = ["GET", "POST"])
def input_page():
    errors = ""
    output = ""
    if request.method == "POST":
        url = None
        url = f"https://www.{request.form['url']}"
      
        if url is not None:
            try:
                
                def generate():
                    yield "<br/>"
                    yield str(crawler(url))
                    links = crawler(url)
                    yield str(kw_ranking(links))
                    output = kw_ranking(links)  
                return Response(generate(), mimetype='text\html')
                                           
                #render_template("results.html", result=output)
            
            except Exception as e:
                #errors = "🐸 The anti-bug frog has caught something! Can you try again with another site? Sorry, it's still a beta version!"
                errors = e
    return render_template("main.html", errors=errors)

if __name__ == '__main__':
    app.run(use_reloader=False)

In [ ]:
#questa versione ora funziona, ma ritorna un risultato plain text veramente brutto. Alternativa scaricare un csv che non è male come cosa;
#ora ho una velocità che si misura in secondi e non in minuti W-O-W! adesso devo solo capire come rendere l'ouput gradevole alla vista e ci siamo! 

from flask import Flask, request, render_template, Response
from concurrent.futures import ThreadPoolExecutor, as_completed
from processing import kw_ranking, crawler, cleaner

app = Flask(__name__)
app.config["DEBUG"] = True


#render about page 
@app.route('/about', methods = ["GET"])
def about():
    return render_template("about.html")

#render main page
@app.route('/', methods = ["GET", "POST"])
def input_page():
    errors = ""
    output = ""
    if request.method == "POST":
        url = None
        url = f"https://www.{request.form['url']}"
      
        if url is not None:
            try:
                links = crawler(url)
                
                def generate_kw(links):
                    MAX_THREADS = 50
                    threads = min(MAX_THREADS, len(links))
                    with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
                        futures = {executor.submit(kw_ranking, link): link for link in links}
                        for result in as_completed(futures):
                            link = futures.get(result)
                            try:
                                data=result.result()
                            except Exception as e:
                                print (e)
                            else:
                                yield (f"Link: {link}\n Data:{data}\n\n")
                #return app.response_class(generate_kw(links), mimetype="text/csv") this download a csv file and it's not bad
                return app.response_class(generate_kw(links), mimetype="html") 
                
                                          
            
            except Exception as e:
                #errors = "🐸 The anti-bug frog has caught something! Can you try again with another site? Sorry, it's still a beta version!"
                errors = e
    return render_template("main.html", errors=errors)

if __name__ == '__main__':
    app.run(use_reloader=False)

In [ ]:
#LAST: si lavora per rendere gradevole l'output in streaming

from flask import Flask, request, render_template, Response
from concurrent.futures import ThreadPoolExecutor, as_completed
from processing import kw_ranking, crawler, cleaner

app = Flask(__name__)
app.config["DEBUG"] = True


#render about page 
@app.route('/about', methods = ["GET"])
def about():
    return render_template("about.html")

#render main page
@app.route('/content', methods = ["GET", "POST"])
def input_page():
    url = None
    links = ""
    if request.method == "POST":
        url = f"https://www.{request.form['url']}"
    if url is not None:
        try:
            links = crawler(url)
            def generate_kw(links):
                MAX_THREADS = 50
                threads = min(MAX_THREADS, len(links))
                with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
                    futures = {executor.submit(kw_ranking, link): link for link in links}
                    for result in as_completed(futures):
                        link = futures.get(result)
                        print ("banana")
                        try:
                            data=result.result()
                        except Exception as e:
                        else:
                            yield (f"Link: {link}\n Data:{data}\n\n")
                            
            return Response(generate_kw(links), mimetype="text/html") 
                        
        except Exception as e:
            return app.response_class(" ", mimetype="text/html")
    else:
        return app.response_class(" ", mimetype="text/html") 
     

@app.route('/', methods = ["GET", "POST"])
def index():
    errors = ""
    output = ""
    
    if request.method == "POST":
        url = None
        url = f"https://www.{request.form['url']}"
      
        if url is not None:
            try:
                input_page()
            except Exception as e:
                error = e 
               
    return render_template('index.html.jinja') 

if __name__ == '__main__':
    app.run(use_reloader=False)

In [ ]:
#questa funziona perfettamente, ma perchè la mia no? 

from flask import Flask, render_template, Response
import time, math

app = Flask(__name__)

@app.route('/content') # render the content a url differnt from index
def content():
    def inner():
        # simulate a long process to watch
        for i in range(500):
            j = math.sqrt(i)
            time.sleep(1)
            # this value should be inserted into an HTML template
            yield str(i) + '<br/>\n'
    return Response(inner(), mimetype='text/html')

@app.route('/')
def index():
    return render_template('test.html.jinja') # render a template at the index. The content will be embedded in this template

if __name__ == '__main__':
    app.run(use_reloader=False)

In [ ]:
#esperimenti, non streamma una mazza


from flask import Flask, request, render_template, Response
from concurrent.futures import ThreadPoolExecutor, as_completed
from processing import kw_ranking, crawler, cleaner

app = Flask(__name__)
app.config["DEBUG"] = True


#render about page 
@app.route('/about', methods = ["GET"])
def about():
    return render_template("about.html")

#render main page
@app.route('/content', methods = ["GET", "POST"])
def input_page():
    url = None
    links = ""
    if request.method == "POST":
        url = f"https://www.{request.form['url']}"
    if url is not None:
        print ("armadillo")
        try:
            links = crawler(url)
            print (links)
            def generate_kw(links_input):
                MAX_THREADS = 50
                threads = min(MAX_THREADS, len(links_input))
                with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
                    futures = {executor.submit(kw_ranking, link): link for link in links_input}
                    for result in as_completed(futures):
                        link_list = futures.get(result)
                        print ("banana")
                        try:
                            data=result.result()
                            print ("uva")
                        except Exception as e:
                            print (e)
                        else:
                            print (link)
                            print (data)
                            print (e)
                            yield (f"Link: {link_list}\n Data:{data}\n\n")
                            
            print ("ok good")
            return Response(generate_kw(links), mimetype="text/html") 
                        
        except Exception as e:
            print (e)
            return app.response_class(" ", mimetype="text/html")
    else:
        print ("ok")
        return app.response_class(" ", mimetype="text/html") 
     


@app.route('/', methods = ["GET", "POST"])
def index():
    errors = ""
    output = ""
    
    if request.method == "POST":
        url = None
        url = f"https://www.{request.form['url']}"
      
        if url is not None:
            try:
                input_page()
            except Exception as e:
                error = e 
               
    return render_template('index.html.jinja') 

if __name__ == '__main__':
    app.run(use_reloader=False)

In [ ]:
def generate_kw(links_input):
                MAX_THREADS = 50
                threads = min(MAX_THREADS, len(links_input))
                with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
                    futures = {executor.submit(kw_ranking, link): link for link in links_input}
                    for result in as_completed(futures):
                        link_list = futures.get(result)
                        print ("banana")
                        try:
                            data=result.result()
                            print ("uva")
                        except Exception as e:
                            print (e)
                        else:
                            yield (f"Link: {link_list}\n Data:{data}\n\n")

In [ ]:
links= {'https://www.cucinaconsapevole.it/2019/12/', 'https://www.cucinaconsapevole.it/risorse/', 'https://www.cucinaconsapevole.it/2018/12/', 'https://www.cucinaconsapevole.it/2019/09/', 'https://www.cucinaconsapevole.it/crumble-di-pere-e-cioccolato/', 'https://www.cucinaconsapevole.it/newsletter', 'https://www.cucinaconsapevole.it/newsletter/', 'https://www.cucinaconsapevole.it/blog/', 'https://www.cucinaconsapevole.it/gazpacho-di-zucchine/', 'https://www.cucinaconsapevole.it/chi-sono/', 'https://www.cucinaconsapevole.it/', 'https://www.cucinaconsapevole.it/menu-veloce-di-natale/', 'https://www.cucinaconsapevole.it/ricette/'}

In [ ]:
generate_kw(links)

for i in generate_kw(links):
    print (i)

In [ ]:
#ultima ipotesi dove si rimette tutto in discussione (cazzo porco ora funziona e streamma valori ce l'ho fatta porco zio dopo letteralmente una settimana di tentativi)
#ora il processo funziona! ma non trovando URL in GET, si pianta (poi inserendo url parte), lavoriamo su questa parte.

import time
from flask import Flask, request, render_template, Response
from concurrent.futures import ThreadPoolExecutor, as_completed
from processing import kw_ranking, crawler, cleaner

app = Flask(__name__)
app.config["DEBUG"] = True

def generate_kw(links_input):
    MAX_THREADS = 50
    threads = min(MAX_THREADS, len(links_input))
    with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        futures = {executor.submit(kw_ranking, link): link for link in links_input}
        for result in as_completed(futures):
            link_list = futures.get(result)
            try:
                data=result.result()
            except Exception as e:
                return "NOK1"
            else:
                yield ""  #this is to "immediatly" return a value
                yield (f"Link: {link_list}\n<br> Data:{data}\n<br><br>")



#render about page 
@app.route('/about', methods = ["GET"])
def about():
    return render_template("about.html")

#render main page
@app.route('/content', methods = ["GET", "POST"])
def input_page():
    #if url is not None:
    while url is not None:
        time.sleep(1) #avoid out of sync actions (not working)
        try:
            global links
            links = crawler(url)
            return Response(generate_kw(links), mimetype="text/html") 

        except Exception as e:
            return ""
    

@app.route('/', methods = ["GET", "POST"])
def index():
    if request.method == "POST":
        global url
        url = f"https://www.{request.form['url']}"
    elif request.method == "GET":
        url = "blankwebsite.com/"
    return render_template('index.html.jinja') 

if __name__ == '__main__':
    app.run(use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jul/2021 12:55:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2021 12:55:59] "GET /content HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2021 12:56:01] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2021 12:56:07] "GET /content HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2021 12:56:19] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2021 12:56:23] "GET /content HTTP/1.1" 200 -
